# Use inversion framework to fit a linear regressor

In [1]:
import numpy as np
from regressor import LinearRegressor

from inversion_ideas import ConjugateGradient, DataMisfit, TikhonovZero

In [2]:
n_params = 10
rng = np.random.default_rng(seed=4242)
true_model = rng.uniform(size=10)
true_model

array([0.78225148, 0.67148671, 0.2373809 , 0.17946133, 0.34662367,
       0.15210999, 0.31142952, 0.23900652, 0.54355731, 0.91770851])

In [3]:
# Build the X array
n_data = 25
shape = (n_data, n_params)
X = rng.uniform(size=n_data * n_params).reshape(shape)

In [4]:
synthetic_data = X @ true_model
maxabs = np.max(np.abs(synthetic_data))
noise = rng.normal(scale=1e-2 * maxabs, size=synthetic_data.size)
synthetic_data += noise
synthetic_data

array([2.83840696, 2.18091081, 2.00623242, 2.08333039, 2.01694883,
       2.7826232 , 2.10564027, 1.27333506, 2.08859855, 1.94177648,
       1.88492037, 2.92394733, 2.17231952, 3.08009275, 1.61670886,
       1.77403753, 2.67305005, 1.91413882, 2.42117827, 2.13991628,
       2.0153805 , 2.71388471, 2.65944255, 2.44416121, 3.14217523])

In [5]:
simulation = LinearRegressor(X)

In [6]:
uncertainty = 1e-2 * maxabs * np.ones_like(synthetic_data)
data_misfit = DataMisfit(synthetic_data, uncertainty, simulation)
smallness = TikhonovZero(n_params)

In [7]:
phi = data_misfit + 1e-3 * smallness
phi

φd(m) + 0.00 φs(m)

In [8]:
initial_model = np.zeros(n_params)
initial_model

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

## Minimize manually with `scipy.sparse.linalg.cg`

In [9]:
from scipy.sparse.linalg import cg

In [10]:
grad = phi.gradient(initial_model)
hess = phi.hessian(initial_model)

In [11]:
model_step, info = cg(hess, -grad)
model_step, info

(array([0.81328886, 0.65927514, 0.2472937 , 0.19624752, 0.3237346 ,
        0.14720342, 0.3194468 , 0.25235983, 0.52215485, 0.92181018]),
 0)

In [12]:
inverted_model = initial_model + model_step

In [13]:
print("Result:")
print(inverted_model)
print()
print("True model:")
print(true_model)

Result:
[0.81328886 0.65927514 0.2472937  0.19624752 0.3237346  0.14720342
 0.3194468  0.25235983 0.52215485 0.92181018]

True model:
[0.78225148 0.67148671 0.2373809  0.17946133 0.34662367 0.15210999
 0.31142952 0.23900652 0.54355731 0.91770851]


## Minimize with SciPy's `minimize`

In [14]:
from scipy.optimize import minimize

In [15]:
result =  minimize(phi, initial_model)
result

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 11.719746150008552
        x: [ 8.133e-01  6.593e-01  2.473e-01  1.963e-01  3.237e-01
             1.472e-01  3.195e-01  2.524e-01  5.222e-01  9.218e-01]
      nit: 15
      jac: [-5.841e-06 -2.384e-07 -3.576e-06 -2.384e-07 -7.987e-06
            -1.311e-06  0.000e+00  1.907e-06 -2.503e-06 -5.603e-06]
 hess_inv: [[ 3.378e-04 -1.043e-04 ...  1.536e-05 -9.322e-05]
            [-1.043e-04  4.045e-04 ... -1.009e-04 -1.335e-05]
            ...
            [ 1.536e-05 -1.009e-04 ...  2.159e-04 -5.467e-05]
            [-9.322e-05 -1.335e-05 ... -5.467e-05  2.625e-04]]
     nfev: 253
     njev: 23

In [16]:
# The minimize already gives you the minimum model
inverted_model = result.x

In [17]:
print("Result:")
print(inverted_model)
print()
print("True model:")
print(true_model)

Result:
[0.8132614  0.65925412 0.24726513 0.19626776 0.32373026 0.14719295
 0.3194738  0.25235066 0.52216838 0.92184153]

True model:
[0.78225148 0.67148671 0.2373809  0.17946133 0.34662367 0.15210999
 0.31142952 0.23900652 0.54355731 0.91770851]


In [18]:
result =  minimize(phi, initial_model, jac=phi.gradient, method="CG")
result

 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: 11.719746150007738
       x: [ 8.133e-01  6.593e-01  2.473e-01  1.963e-01  3.237e-01
            1.472e-01  3.195e-01  2.524e-01  5.222e-01  9.218e-01]
     nit: 12
     jac: [-2.176e-11 -1.929e-11 -1.793e-11 -3.187e-11 -2.110e-11
           -2.648e-11 -2.277e-11 -1.810e-11 -1.589e-11 -1.227e-11]
    nfev: 24
    njev: 24

In [19]:
# The minimize already gives you the minimum model
inverted_model = result.x

In [20]:
print("Result:")
print(result.x)
print()
print("True model:")
print(true_model)

Result:
[0.8132614  0.65925411 0.24726513 0.19626776 0.32373025 0.14719295
 0.3194738  0.25235066 0.52216838 0.92184154]

True model:
[0.78225148 0.67148671 0.2373809  0.17946133 0.34662367 0.15210999
 0.31142952 0.23900652 0.54355731 0.91770851]


In [21]:
result =  minimize(phi, initial_model, jac=phi.gradient, hess=phi.hessian, method="Newton-CG")
result

 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: 11.719746314217307
       x: [ 8.133e-01  6.592e-01  2.473e-01  1.963e-01  3.237e-01
            1.472e-01  3.195e-01  2.524e-01  5.222e-01  9.218e-01]
     nit: 10
     jac: [-3.047e-02 -4.461e-02  1.683e-02  2.203e-02 -8.361e-03
            7.088e-02 -2.080e-02  3.307e-02 -1.339e-02 -1.339e-02]
    nfev: 11
    njev: 11
    nhev: 26

In [22]:
# The minimize already gives you the minimum model
inverted_model = result.x

In [23]:
print("Result:")
print(result.x)
print()
print("True model:")
print(true_model)

Result:
[0.81326764 0.65924856 0.24725963 0.19626682 0.32372755 0.14719478
 0.31947594 0.25235815 0.52216987 0.92183788]

True model:
[0.78225148 0.67148671 0.2373809  0.17946133 0.34662367 0.15210999
 0.31142952 0.23900652 0.54355731 0.91770851]


## Use `Minimizer` class

In [24]:
minimizer = ConjugateGradient()
minimizer

In [25]:
inverted_model = minimizer(phi, initial_model)

In [26]:
print("Result:")
print(inverted_model)
print()
print("True model:")
print(true_model)

Result:
[0.81328886 0.65927514 0.2472937  0.19624752 0.3237346  0.14720342
 0.3194468  0.25235983 0.52215485 0.92181018]

True model:
[0.78225148 0.67148671 0.2373809  0.17946133 0.34662367 0.15210999
 0.31142952 0.23900652 0.54355731 0.91770851]
